# Model fitting and validation
### by [Jason DeBacker](https://jasondebacker.com/), October 2025

This notebook uses machine learning tools in Python's [`scikit-learn`](https://scikit-learn.org/stable/) package to illustrate model fitting and validation.  By validation, I'm referring to tests of the accuracy of the model outside of the sample from which is was fit (i.e., estimated).

## 1. Fitting a model

We begin with the PSID data we used in Problem Set #4.  We'll fit the same model prosed there (dropping the indicator for Hispanic ethnicity since there are not observations in the sample data of that ethnicity):

$$
ln(wage_{i,t}) = \alpha + \beta_1 Educ_{i,t} + \beta_2 Age_{i,t} + \beta_3 Age_{i,t}^2 + \beta_4 Black_{i,t} + \beta_5 OtherRace{i,t} + \varepsilon_{i,t}
$$

Here, we fit the model using the `linear_models` module from the `scikit-learn` package.

In [1]:
# Imports
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn.linear_model import LinearRegression

In [8]:
# Read in data and create variables
data_path = os.path.join('..', 'Optimization', 'PSID_data.dta')
psid = pd.read_stata(data_path,
                     columns=['id68', 'year', 'hannhrs', 'hlabinc', 'hsex',
                              'hyrsed', 'age', 'hrace'])

# create wages and ln(wages)
# note need to be careful with wages = 0
psid['wage'] = psid['hlabinc']/psid['hannhrs']
psid['ln_wage'] = np.log(psid['wage'])

# create age squared
psid['age_sq'] = psid['age'] ** 2

# sample selection
psid.drop(psid[psid.hsex != 1].index, inplace=True)
psid.drop(psid[psid.age > 60].index, inplace=True)
psid.drop(psid[psid.age < 25].index, inplace=True)
psid.drop(psid[psid.wage < 7].index, inplace=True)
psid.drop(psid[psid.wage == np.inf].index, inplace=True)

# create dummy variables for race
psid['black'] = (psid['hrace'] == 2).astype(int)
psid['hispanic'] = (psid['hrace'] == 5).astype(int)
psid['other'] = (
    (psid['hrace'] == 3) | (psid['hrace'] == 4) |
    (psid['hrace'] == 6) | (psid['hrace'] == 7)).astype(int)

# drop obs if missing values for any variabls in regression model
psid.dropna(axis=0,
            subset=['ln_wage', 'hyrsed', 'age', 'age_sq', 'black', 'other'],
            inplace=True)

# add a constant
psid['const'] = 1

# keep just year 2000 so can compare to results from PS #4
psid = psid[psid.year==2000]
psid.shape

(2595, 15)

In [4]:
# Finally, estimate the model
# Create linear regression object
regr = LinearRegression()

# Estimate the model using the full set of PSID data
regr.fit(
    psid[['hyrsed', 'age', 'age_sq', 'black', 'other']],  # the X's, it'll include a constant by default, but can change
    psid['ln_wage']  # the y
)

# Get predicted values
wage_pred = regr.predict(
    psid[['hyrsed', 'age', 'age_sq', 'black', 'other']])

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % sklearn.metrics.mean_squared_error(
          psid['ln_wage'],  #actual values
          wage_pred)  # predicted values
     )
# The R-sq:
print('R-squared: %.2f'
      % sklearn.metrics.r2_score(
          psid['ln_wage'],  #actual values
          wage_pred)  # predicted values
     )

Coefficients: 
 [ 0.11033282  0.0843604  -0.00088677 -0.25961741 -0.0621439 ]
Mean squared error: 0.29
R-squared: 0.22


## 2. Model Validation

Now we'll consider some approaches to model validation.  That is, we'll see if our parameter estimates provide to give us accurate predictions, even on data we didn't train the model with.

Recall from our lectures, we discussed 3 validation approaches:
1. Validation data
2. Leave on out cross-validation
3. K-fold cross-validation

We'll consider each of these in turn below.

### 2.1. Validation data

Validation data are data we set aside in order to test out model.  We do this as follows:

1. Partition the data into a training set and a test set.
2. Estimate the model using the training set.
3. Evaluate the fit or predictive accuracy on the test set.

The primary measure of fit is the mean squared error (MSE) of the estimated model on the test set. Let the test set have $n$ observations. The MSE of the test set is the sum of squared deviations of the actual dependent variable values minus the predicted values.

$$
MSE_{test} = \frac{1}{n}\sum_{i=1}^n\left(y_i - \hat{y}_i\right)^2
$$

Let's employ the validation data approach on our PSID data.

In [5]:
# Step 1: partition the data
# This function train_test_split is from sklearn.cross_validation
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    psid[['hyrsed', 'age', 'age_sq', 'black', 'other']],   # the X's
    psid['ln_wage'],  # the y's
    test_size=0.4,  # use 60% of data to train, 40% to test
    random_state=25  # can set the seed so can reproduce results
)

In [6]:
# Step 2: estimate the model on the training data
# Create linear regression object
regr = LinearRegression()
# Estimate the model
regr.fit(
    X_train,  # the X's, it'll include a constant by default, but can change
    y_train  # the y
)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [7]:
# Evaluate the accuracy of the model on
# the validation data (and on the training
# data for comparison)

# get predicted values on training data
pred_train = regr.predict(X_train)

# get predicted values on test data
pred_test = regr.predict(X_test)

# The mean squared error
print('Mean squared error on the training data: %.2f'
      % sklearn.metrics.mean_squared_error(
          y_train,  #actual values
          pred_train)  # predicted values
     )
print('Mean squared error on the test data: %.2f'
      % sklearn.metrics.mean_squared_error(
          y_test,  #actual values
          pred_test)  # predicted values
     )

Mean squared error on the training data: 0.28
Mean squared error on the test data: 0.30


Unsurprisingly, our fit is better on the training data than the test data!  But not by much.

### 2.2. Leave-one-out cross validation

Leave-one-out cross validation (LOOCV) were we create $n$ data sets each of length $n-1$ by dropping a single observation from our full dataset (once for each observation).  We estimate the model on the $n-1$ observations and then compute the squared error when we try to predict the dropped observation with our fitted model:

$$
SE_n = \left(y_n - \hat{f}_n(x_n)\right)^2
$$

The LOOCV estimate for the test MSE is the average of these $n$ squared errors:

$$
MSE_{test} = \frac{1}{n}\sum_{i=1}^n SE_i
$$

In [9]:
# create separate X and y so don't repeat DataFrame slices many times below
Xvars = psid[['hyrsed', 'age', 'age_sq', 'black', 'other']].values
yvars = psid['ln_wage'].values
loo = sklearn.model_selection.LeaveOneOut()
loo.get_n_splits(Xvars)

# Create array to fill with squared errors
n = psid.shape[0]
SE_test = np.zeros(n)

# Loop over all observations using the LeaveOneOut split method
for train_index, test_index in loo.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvars[train_index], yvars[test_index]
    regr = sklearn.linear_model.LinearRegression()
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    SE_test[test_index] = (y_test - y_pred) ** 2

MSE_test = SE_test.mean()
MSE_test_std = SE_test.std()
print('LOOCV test MSE =', MSE_test)
print('Std dev of the LOOCV test MSE =', MSE_test_std)

LOOCV test MSE = 0.28667286682387183
Std dev of the LOOCV test MSE = 0.5916542489044918


We can see the test MSE is lower here than the validation data approach.  This is because we are using more data to train our model.


### 2.3. k-fold cross validation

$k$-fold cross validation is a method in which the dataset is randomly divided into $k$ groups (folds). We then set aside one of the folds to test our data on and use the observations in the other $k-1$ folds to train our data.  We then repeat this process $K$ times, leaving each of the $K$ folds out of the fiting process once.  Let $MSE_k$ be the MSE from the $k$th fold:

$$
MSE_{k} = \frac{1}{n/k}\sum_{j=1}^{n/k}(y_{j} - \hat{f}_{k}(x_{j}))^2
$$

Then the $k$-fold estimate for the test MSE is the average of these $k$ test error estimates.

$$
MSE_{test} = \frac{1}{k}\sum_{i=1}^k MSE_i
$$

LOOCV is a special case of $k$-fold cross validation in which $k=n$.

Let's implement this in Python:

Let's use the PSID data again and test our logit model performance with a $k$-fold cross validation with $k=6$.

In [13]:
k = 4  # number of folds
kf = sklearn.model_selection.KFold(
    n_splits=k,
    random_state=10,  # set seed so can reproduce results
    shuffle=True
)
kf.get_n_splits(Xvars)

# initialize array to fill with MSEs
MSE_k = np.zeros(k)

# Loop over the splits
k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvars[train_index], yvars[test_index]
    regr = sklearn.linear_model.LinearRegression()
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    MSE_k[k_ind] = ((y_test - y_pred) ** 2).mean()
    k_ind += 1

MSE_test = MSE_k.mean()
MSE_test_std = MSE_k.std()
print('LOOCV test MSE =', MSE_test)
print('Std dev of the LOOCV test MSE =', MSE_test_std)

LOOCV test MSE = 0.28660046801734734
Std dev of the LOOCV test MSE = 0.007104389822942944


### 2.4. Bias versus variance
Recall the test estimate MSE from the LOOCV of approximately 0.2868 and the MSE(LOOCV) standard error of about 0.0444. What happens to the estimated MSE and MSE standard error in the $k$-fold cross validation above as $k$ increases? Try values of $k=2, 10, 50, 100, 800$.

Note that the LOOCV method has low bias (estimated on large number of data) but high variance (errors are based on one draw). In contrast, the $k$-fold method has more bias (estimated with less data) but lower variance. Each test set has more observations.

* $k$-fold cross validation can often provide more accurate estimates of the test error rate.
* $k$-fold is less computationally intensive
* LOOCV has the least bias
* LOOCV is the most computationally expensive